In [1]:
!pip install torch torchvision torchaudio
!pip install roboflow
!pip install matplotlib numpy
!pip install roboflow
!pip install split-folders
!pip install split-folders
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu126



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import splitfolders
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader

In [11]:
from roboflow import Roboflow
rf = Roboflow(api_key="AWN4nVlJhx52ROuzjIPX")
project = rf.workspace("itrash").project("apple-khthj")
version = project.version(4)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Apple-4 in multiclass:: 100%|██████████| 3383/3383 [00:05<00:00, 575.75it/s]


In [ ]:
# ==============================
# 1. Custom Dataset Class
# ==============================
class AppleRipenessDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.class_names = list(self.annotations.columns[1:])  # skip filename col

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")

        labels = self.annotations.iloc[idx, 1:].values.astype("float")
        label = torch.tensor(labels).argmax().long()

        if self.transform:
            image = self.transform(image)

        return image, label

# ==============================
# 2. Setup
# ==============================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_dir = "Apple-4"  # adjust if different

data_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

datasets_custom = {
    split: AppleRipenessDataset(
        csv_file=os.path.join(data_dir, split, "_classes.csv"),
        img_dir=os.path.join(data_dir, split),
        transform=data_transforms
    )
    for split in ["train", "valid", "test"]
}

dataloaders = {
    split: DataLoader(datasets_custom[split], batch_size=32, shuffle=True, num_workers=2)
    for split in ["train", "valid", "test"]
}

dataset_sizes = {x: len(datasets_custom[x]) for x in ["train", "valid", "test"]}
class_names = datasets_custom["train"].class_names
print("Classes:", class_names)

# ==============================
# 3. Load Pretrained ResNet
# ==============================
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ==============================
# 4. Training Function
# ==============================
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "valid"]:
            model.train() if phase == "train" else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == "valid" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model

# ==============================
# 5. Train
# ==============================
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)

# ==============================
# 6. Test Accuracy
# ==============================
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in dataloaders["test"]:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Using device: cpu
Classes: ['apple_ripe', 'apple_rotten', 'apple_unripe']


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\naval/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:23<00:00, 1.99MB/s]


Epoch 1/10
----------
